In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("spark://spark-master:7077").getOrCreate()
sc = spark.sparkContext

rdd = sc.parallelize(range(100 + 1))
rdd.sum()

5050

In [16]:
df = spark.createDataFrame(
    [
        ("Tony Minels", 18),
        ("Vincent Pres", 20),
        ("Valter Panat", 12),
        ("Luca Mosca", 25),
        ("Jean", 30),
        ("Pierino", 11)        
    ],
    ["first_name", "age"]
)

In [17]:
df.show()

+------------+---+
|  first_name|age|
+------------+---+
| Tony Minels| 18|
|Vincent Pres| 20|
|Valter Panat| 12|
|  Luca Mosca| 25|
|        Jean| 30|
|     Pierino| 11|
+------------+---+



In [7]:
from pyspark.sql.functions import col, when

df1 = df.withColumn(
    "life_stage",
    when(col("age") < 13, "child")
    .when(col("age").between(13,19), "teenager")
    .otherwise("adult")
)

In [8]:
df1.show()

+------------+---+----------+
|  first_name|age|life_stage|
+------------+---+----------+
| Tony Minels| 18|  teenager|
|Vincent Pres| 20|     adult|
|Valter Panat| 12|     child|
+------------+---+----------+



In [9]:
df.show()

+------------+---+
|  first_name|age|
+------------+---+
| Tony Minels| 18|
|Vincent Pres| 20|
|Valter Panat| 12|
+------------+---+



In [11]:
df1.where( col("life_stage").isin(["teenager", "adult"]) ).show()

+------------+---+----------+
|  first_name|age|life_stage|
+------------+---+----------+
| Tony Minels| 18|  teenager|
|Vincent Pres| 20|     adult|
+------------+---+----------+



In [12]:
from pyspark.sql.functions import avg

df1.select(avg("age")).show()

+------------------+
|          avg(age)|
+------------------+
|16.666666666666668|
+------------------+



In [14]:
spark.sql("select avg(age) from {df1}", df1=df1).show()

+------------------+
|          avg(age)|
+------------------+
|16.666666666666668|
+------------------+



In [18]:
spark.sql("select life_stage, avg(age) from {df1} group by life_stage", df1=df1).show()

+----------+--------+
|life_stage|avg(age)|
+----------+--------+
|     adult|    20.0|
|  teenager|    18.0|
|     child|    12.0|
+----------+--------+



In [19]:
df1.write.saveAsTable("peoples")

Py4JJavaError: An error occurred while calling o161.saveAsTable.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 37.0 failed 4 times, most recent failure: Lost task 0.3 in stage 37.0 (TID 777) (172.19.0.5 executor 0): java.io.IOException: Mkdirs failed to create file:/home/jovyan/spark-warehouse/peoples/_temporary/0/_temporary/attempt_202506271412358828699375140776092_0037_m_000000_777 (exists=false, cwd=file:/opt/bitnami/spark/work/app-20250627135103-0000/0)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:715)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:700)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1233)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1210)
	at org.apache.parquet.hadoop.util.HadoopOutputFile.create(HadoopOutputFile.java:82)
	at org.apache.parquet.hadoop.ParquetFileWriter.<init>(ParquetFileWriter.java:473)
	at org.apache.parquet.hadoop.ParquetFile